<a href="https://colab.research.google.com/github/BooEiEi/Predicting-Technology-Sector-Stock-Prices/blob/main/Hyperparameter_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF4'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sklearn
#import scikeras
import tensorflow

print("scikit-learn version:", sklearn.__version__)
#print("scikeras version:", scikeras.__version__)
print("tensorflow version:", tensorflow.__version__)


scikit-learn version: 1.5.2
tensorflow version: 2.18.0


In [3]:
!pip uninstall scikit-learn -y

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1


In [4]:
pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 118.5 MB/s eta 0:00:00


In [ ]:
import os
os._exit(00)

In [3]:
!pip install scikeras

In [4]:
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from keras.layers import Dense, GRU
from tensorflow.keras.layers import Dense, Flatten, LSTM, Bidirectional, Input, Dropout, InputLayer, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [10]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF, 'df_GOOG.csv'))

In [11]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])
df_META = df_META.sort_values('date').reset_index(drop=True)

In [12]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
df_AAPL = df_AAPL.sort_values('date').reset_index(drop=True)

In [13]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
df_MSFT = df_MSFT.sort_values('date').reset_index(drop=True)

In [14]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
df_NVDA = df_NVDA.sort_values('date').reset_index(drop=True)

In [15]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])
df_GOOG = df_GOOG.sort_values('date').reset_index(drop=True)

In [16]:
train_size = int(len(df_META) * 0.75)
test_size = len(df_META) - train_size

# META

## Train Test

In [17]:
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

### Train Varidation

In [18]:
X_META = train_META[["S&P500","CPI","Interest","US_inflation_rate", "NASDAQ"]]
y_META = train_META["Stockprice_META"]

#Normalized

In [19]:
y_META

,Stockprice_META
0,203.49
1,208.11
2,202.97
3,200.06
4,187.47
...,...
559,464.63
560,467.78
561,465.78
562,478.22


In [20]:
train_META

,date,Stockprice_META,sentiment_META,Interest,UNEMPLOYMENT,CPI,US_Treasury_Yield,RETAIL_SALES,US_inflation_rate,Real_GDP,US_Dollar_Index,S&P500,NASDAQ,WTI,BTC,Gold
0,2022-03-01,203.49,0.000000,0.08,3.7,287.504,1.72,595064.0,0.075,5337.277,97.41,4306.26,13532.46,107.77,44420.3,1943.8
1,2022-03-02,208.11,0.000000,0.08,3.7,287.504,1.86,595064.0,0.075,5337.277,97.39,4386.54,13752.02,112.22,43912.8,1922.3
2,2022-03-03,202.97,0.000000,0.08,3.7,287.504,1.86,595064.0,0.075,5337.277,97.79,4363.49,13537.94,110.52,42463.0,1935.9
3,2022-03-04,200.06,0.000000,0.08,3.7,287.504,1.74,595064.0,0.075,5337.277,98.65,4328.87,13313.44,116.37,39142.7,1966.6
4,2022-03-07,187.47,0.000000,0.08,3.7,287.504,1.78,595064.0,0.075,5337.277,99.29,4201.09,12830.96,122.09,38024.8,1995.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2024-05-21,464.63,0.129250,5.33,4.0,314.069,4.41,643843.0,0.034,5817.169,104.66,5321.41,16832.62,78.69,70139.9,2425.9
560,2024-05-22,467.78,0.622400,5.33,4.0,314.069,4.43,643843.0,0.034,5817.169,104.93,5307.01,16801.54,77.53,69155.4,2392.9
561,2024-05-23,465.78,0.399033,5.33,4.0,314.069,4.47,643843.0,0.034,5817.169,105.11,5267.84,16736.03,77.22,67975.7,2337.2
562,2024-05-24,478.22,0.437117,5.33,4.0,314.069,4.46,643843.0,0.034,5817.169,104.72,5304.72,16920.79,78.18,68547.6,2334.5


In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_META = scaler.fit_transform(X_META)

In [22]:
X_META

array([[0.41804538, 0.        , 0.        , 0.73770492, 0.48764065],
       [0.46406746, 0.        , 0.        , 0.73770492, 0.51989763],
       [0.4508536 , 0.        , 0.        , 0.73770492, 0.48844576],
       ...,
       [0.96928995, 1.        , 1.        , 0.06557377, 0.95829777],
       [0.99043213, 1.        , 1.        , 0.06557377, 0.98544205],
       [0.99118885, 1.        , 1.        , 0.06557377, 1.        ]])

In [23]:
X_META.shape[0]

564

In [24]:
look_back = 10

X_META1, y_META1 = [], []
for i in range(len(X_META) - look_back):
    X_META1.append(X_META[i:i + look_back])
    y_META1.append(y_META[i + look_back])

X_META1, y_META1 = np.array(X_META1), np.array(y_META1)

In [25]:
y_META1.shape

(554,)

In [26]:
X_META1.shape

(554, 10, 5)

In [27]:
tscv = TimeSeriesSplit(n_splits=5)

#MODEL

In [33]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(GRU(units=neurons, input_shape=(look_back, X_META1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_META1, y_META1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_


y_pred = best_model.predict(X_META1)
mse = mean_squared_error(y_META1, y_pred)
rmse = root_mean_squared_error(y_META1, y_pred)
mae = mean_absolute_error(y_META1, y_pred)
mape = mean_absolute_percentage_error(y_META1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 150, 'neurons': 512, 'optimizer': 'RMSprop'}
Test Set Mean Squared Error: 156.6028
Test Set Root Mean Squared Error: 12.5141
Test Set Mean Absolute Error: 9.1056
Test Set Mean Absolute Percentage Error: 0.0392


#APPL

In [34]:
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [35]:
X_AAPL = train_AAPL[["S&P500","CPI", "NASDAQ"]]
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [36]:
X_AAPL = scaler.fit_transform(X_AAPL)

In [37]:
look_back = 10

X_AAPL1, y_AAPL1 = [], []
for i in range(len(X_AAPL) - look_back):
    X_AAPL1.append(X_AAPL[i:i + look_back])
    y_AAPL1.append(y_AAPL[i + look_back])

X_AAPL1, y_AAPL1 = np.array(X_AAPL1), np.array(y_AAPL1)

In [38]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(GRU(units=neurons, input_shape=(look_back, X_AAPL1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_AAPL1, y_AAPL1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_AAPL1)
mse = mean_squared_error(y_AAPL1, y_pred)
rmse = root_mean_squared_error(y_AAPL1, y_pred)
mae = mean_absolute_error(y_AAPL1, y_pred)
mape = mean_absolute_percentage_error(y_AAPL1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'neurons': 512, 'optimizer': 'Nadam'}
Test Set Mean Squared Error: 14.2868
Test Set Root Mean Squared Error: 3.7798
Test Set Mean Absolute Error: 3.0135
Test Set Mean Absolute Percentage Error: 0.0182


#MSFT

In [39]:
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [40]:
X_MSFT = train_MSFT[["S&P500","CPI","Interest","US_inflation_rate", "NASDAQ"]]
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [41]:
X_MSFT = scaler.fit_transform(X_MSFT)

In [42]:
look_back = 10

X_MSFT1, y_MSFT1 = [], []
for i in range(len(X_MSFT) - look_back):
    X_MSFT1.append(X_MSFT[i:i + look_back])
    y_MSFT1.append(y_MSFT[i + look_back])

X_MSFT1, y_MSFT1 = np.array(X_MSFT1), np.array(y_MSFT1)

In [43]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(GRU(units=neurons, input_shape=(look_back, X_MSFT1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_MSFT1, y_MSFT1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_MSFT1)
mse = mean_squared_error(y_MSFT1, y_pred)
rmse = root_mean_squared_error(y_MSFT1, y_pred)
mae = mean_absolute_error(y_MSFT1, y_pred)
mape = mean_absolute_percentage_error(y_MSFT1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'neurons': 512, 'optimizer': 'Adam'}
Test Set Mean Squared Error: 143.7482
Test Set Root Mean Squared Error: 11.9895
Test Set Mean Absolute Error: 9.3632
Test Set Mean Absolute Percentage Error: 0.0303


#NVDA

In [44]:
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [45]:
X_NVDA = train_NVDA[["S&P500", "BTC", "CPI", "Gold","US_inflation_rate", "NASDAQ"]]
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [46]:
X_NVDA = scaler.fit_transform(X_NVDA)

In [47]:
look_back = 10

X_NVDA1, y_NVDA1 = [], []
for i in range(len(X_NVDA) - look_back):
    X_NVDA1.append(X_NVDA[i:i + look_back])
    y_NVDA1.append(y_NVDA[i + look_back])

X_NVDA1, y_NVDA1 = np.array(X_NVDA1), np.array(y_NVDA1)

In [48]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(GRU(units=neurons, input_shape=(look_back,X_NVDA1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_NVDA1, y_NVDA1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_NVDA1)
mse = mean_squared_error(y_NVDA1, y_pred)
rmse = root_mean_squared_error(y_NVDA1, y_pred)
mae = mean_absolute_error(y_NVDA1, y_pred)
mape = mean_absolute_percentage_error(y_NVDA1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'neurons': 512, 'optimizer': 'RMSprop'}
Test Set Mean Squared Error: 6.4995
Test Set Root Mean Squared Error: 2.5494
Test Set Mean Absolute Error: 1.6196
Test Set Mean Absolute Percentage Error: 0.0482


#GOOG

In [49]:
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [50]:
X_GOOG = train_GOOG[["S&P500", "Gold", "CPI", "Interest", "UNEMPLOYMENT", "NASDAQ"]]
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [51]:
X_GOOG = scaler.fit_transform(X_GOOG)

In [52]:
look_back = 10

X_GOOG1, y_GOOG1 = [], []
for i in range(len(X_GOOG) - look_back):
    X_GOOG1.append(X_GOOG[i:i + look_back])
    y_GOOG1.append(y_GOOG[i + look_back])

X_GOOG1, y_GOOG1 = np.array(X_GOOG1), np.array(y_GOOG1)

In [53]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(GRU(units=neurons, input_shape=(look_back,X_GOOG1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_GOOG1, y_GOOG1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_GOOG1)
mse = mean_squared_error(y_GOOG1, y_pred)
rmse = root_mean_squared_error(y_GOOG1, y_pred)
mae = mean_absolute_error(y_GOOG1, y_pred)
mape = mean_absolute_percentage_error(y_GOOG1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'neurons': 256, 'optimizer': 'AdamW'}
Test Set Mean Squared Error: 15.3281
Test Set Root Mean Squared Error: 3.9151
Test Set Mean Absolute Error: 3.0910
Test Set Mean Absolute Percentage Error: 0.0260
